# Классификация комментариев пользователей интернет-магазина

## Описание исследования

В новом сервисе интернет-магазина пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. Магазину требуется инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Для этого необходимо решить задачу бинарной классификации и обучить модель для классификации комментариев на позитивные и негативные на основе набора данных с разметкой о токсичности правок (обучение с учителем).

Качество модели проверяется с помощью метрики *F1*, значение которой на тестовой выборке должно быть не меньше 0.75.

## Цель исследования

Целью исследования является построение модели для классификации комментариев пользователей на позитивные и негативные со значением метрики качества *F1* не меньше 0.75

## Задачи исследования

- Загрузка и первичное знакомство с данными;
- Предобработка данных, включающая очистку текста от незначимых символов;
- Подготовка текстов для обучения моделей: токенизация, лемматизация и векторизация с помощью `TfidfVectorizer`;
- Обучение с учителем следующих моделей:
 - `CatBoostClassifier`;
 - `RandomForestClassifier`;
 - `LogisticRegression`.
- Выбор лучшей модели и проверка её качества на тестовой выборке.

## Исходные данные

Входной признак:

`text` — текст комментария.


Целевой признак:

`toxic` — класс, к которому относится комментарий: `0`- нетоксичный, `1` - токсичный.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-исследования" data-toc-modified-id="Описание-исследования-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание исследования</a></span></li><li><span><a href="#Цель-исследования" data-toc-modified-id="Цель-исследования-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Цель исследования</a></span></li><li><span><a href="#Задачи-исследования" data-toc-modified-id="Задачи-исследования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Задачи исследования</a></span></li><li><span><a href="#Исходные-данные" data-toc-modified-id="Исходные-данные-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Исходные данные</a></span></li><li><span><a href="#Импорт-библиотек-и-знакомство-с-данными" data-toc-modified-id="Импорт-библиотек-и-знакомство-с-данными-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Импорт библиотек и знакомство с данными</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Подготовка-данных-для-обучения-моделей" data-toc-modified-id="Подготовка-данных-для-обучения-моделей-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Подготовка данных для обучения моделей</a></span></li><li><span><a href="#Обучение-с-учителем.-Задача-бинарной-классификации" data-toc-modified-id="Обучение-с-учителем.-Задача-бинарной-классификации-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Обучение с учителем. Задача бинарной классификации</a></span><ul class="toc-item"><li><span><a href="#Обучение-модели-CatBoostClassifier" data-toc-modified-id="Обучение-модели-CatBoostClassifier-8.1"><span class="toc-item-num">8.1&nbsp;&nbsp;</span>Обучение модели CatBoostClassifier</a></span></li><li><span><a href="#Обучение-модели-RandomForestClassifier" data-toc-modified-id="Обучение-модели-RandomForestClassifier-8.2"><span class="toc-item-num">8.2&nbsp;&nbsp;</span>Обучение модели RandomForestClassifier</a></span></li><li><span><a href="#Обучение-модели-логистической-регрессии" data-toc-modified-id="Обучение-модели-логистической-регрессии-8.3"><span class="toc-item-num">8.3&nbsp;&nbsp;</span>Обучение модели логистической регрессии</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-на-тестовой-выборке-8.4"><span class="toc-item-num">8.4&nbsp;&nbsp;</span>Проверка лучшей модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Импорт библиотек и знакомство с данными

In [1]:
# обновляем библиотеку sklearn
!pip install --upgrade scikit-learn -q

In [2]:
# обновляем библиотеку tqdm
!pip install --upgrade tqdm -q

In [3]:
# загружаем модель для английского языка
!python -m spacy download en_core_web_md -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [4]:
# делаем необходимые импорты
import re

import nltk
import numpy as np
import pandas as pd
import spacy

from nltk.corpus import stopwords as nltk_stopwords

from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

from tqdm import notebook

In [5]:
# объявляем константы
RANDOM_STATE = 2024
TEST_SIZE = 0.25

In [6]:
# загружаем данные
try:
    df_toxic_comments = pd.read_csv(
        'toxic_comments.csv'
    )
except:
    df_toxic_comments = pd.read_csv(
        'https://code.s3.yandex.net/datasets/toxic_comments.csv'
    )

In [7]:
# выводим первые пять строк датафрейма
df_toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
# выводим общую информацию о датафрейме
df_toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [9]:
# смотрим соотношение классов целевого признака в выборке
df_toxic_comments.toxic.value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [10]:
# проверяем уникальные значения классов целевого признака
df_toxic_comments.toxic.unique()

array([0, 1])

**Вывод**

По результатам первичного знакомства с данными можно сделать следующие выводы:

- В выборке содержится около 160 тыс. текстов;
- Пропущенных значений в выборке не наблюдается, однако визуально можно заметить наличие незначимых символов в текстах (например: `\n`), поэтому на следующем этапе необходимо провести очистку текста от всех возможных незначимых символов;
- В выборке наблюдается существенный дисбаланс классов целевого признака: 90% комментариев относятся к классу `0` (нетоксичные), 10% - к классу `1` (токсичные).
- Целевой признак принимает только два уникальных значения (`0` и `1`) и имеет тип `int64`, поэтому дополнительной обработки этой колонки не требуется.
- Первая колонка в выборке (`Unnamed`) не несёт в себе значимой информации для обучения моделей, поэтому не будет использована в дальнейшем.

## Предобработка данных

Выполним очистку текстов от незначимых символов с помощью регулярных выражений:

In [11]:
# форматируем тексты и очищаем их от ненужных символов
df_toxic_comments.text = df_toxic_comments.text.apply(
    lambda x: re.sub('[^a-zA-Z ]', ' ', str(x).lower())
)
df_toxic_comments.text = df_toxic_comments.text.apply(
    lambda x: re.sub(r' +', ' ', x).strip()
)

Выполним простую проверку получившихся текстов на краях отсортированных списков текстов, так как там могли появиться пропуски либо какие-либо ещё неожиданные результаты после очистки:

In [12]:
# сортируем тексты по возрастанию
sorted(df_toxic_comments.text)

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'a according to wikipedia naming conventions the title should be the most common name that isn t offensive to large groups of people b the name is very common as the google results show but i agree that it may be offensive to large groups of people on one side of the wall so maybe it shouldn t be the title but it is a very common name and the page apartheid wall redirects here so it should be listed in the first line as one of the names that the wall is called',
 'a an we generally don t change between british and american english spelling usage unless there is a compelling reason to do so i don t believe that levant british english reaches that standard',
 'a and c the layout and image choice in a appears to be better balanced than b however as someone pointed out above the image has not scaled well to the size displayed in the collapse boxes above so that needs to be improved correct balancing is important and the oversized top 

In [13]:
# сортируем тексты по убыванию
sorted(df_toxic_comments.text, reverse=True)

['zzzzzzzzzzzzzzzzzzzzzzzzzzzzzz thompsma apparently in between your personal attacks on me and your long winded pov pushing here you re not listening to anyone you re quote mining you re pushing a creationist pov and you re getting very tendentious talk contributions',
 'zzzzzzz youre a real bore now go bore someone else tw t',
 'zzzz oregon cotw howdy ya ll time for another collaboration of the week from wikiproject oregon last week we improved flag of oregon detroit lake enough i think to move them to start class so great job everyone this week we have another request in oregon ballot measure and a randomly selected two sentence stub that should be easy to expand enough for a dyk in wallowa whitman national forest to opt out of these messages leave your name here or click here to make a suggestion',
 'zzuuzz is a neo nazi',
 'zydeco experiment was the name and buffalo zydeco is the name that donna the buffalo use when playing zydeco music',
 'zweigenbaum if you can furnish any such 

Мы видим, что после очистки текстов регулярными выражениями появились пустые строки `''`. Удалим их:

In [14]:
# удаляем пустые строки
df_toxic_comments = df_toxic_comments[~(df_toxic_comments['text'] == '')]
df_toxic_comments.reset_index(drop=True, inplace=True)

In [15]:
# выводим общую информацию об обработанном датафрейме
df_toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159281 entries, 0 to 159280
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159281 non-null  int64 
 1   text        159281 non-null  object
 2   toxic       159281 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


**Вывод**

В результате предобработки тексты были очищены от незначимых символов с помощью регулярных выражений. Также из датафрейма были удалены 11 пустых строк, что статистически незначимо для дальнейшего обучения моделей.

Данные готовы для токенизации, лемматизации и векторизации.

## Подготовка данных для обучения моделей

Выполним токенизацию и лемматизацию текстов с помощью языковой модели `spacy` среднего размера для английского языка.

В векторизованную форму переведём тексты с помощью конвертера `TfidfVectorizer`, использующего метод `TF-IDF` для вычисления важности каждого слова в тексте относительно количества его употреблений в данном тексте и во всём корпусе текстов.

In [17]:
# создаём корпус для обучения модели
corpus = list(df_toxic_comments.text)

In [18]:
# задаём токенизатор
nlp = spacy.load('en_core_web_md')

In [19]:
# лемматизируем тексты
corpus_lemm = [' '.join([token.lemma_ for token in text])
               for text in notebook.tqdm(nlp.pipe(corpus,
                                                  n_process=4,
                                                  disable=['parser', 'ner']))]

0it [00:00, ?it/s]

In [20]:
# делаем выборочную проверку лемматизации
print('Исходный текст:\n', corpus[20])
print('Лемматизированный текст:\n', corpus_lemm[20])

Исходный текст:
 regarding your recent edits once again please read wp filmplot before editing any more film articles your edits are simply not good with entirely too many unnecessary details and very bad writing please stop before you do further damage the
Лемматизированный текст:
 regard your recent edit once again please read wp filmplot before edit any more film article your edit be simply not good with entirely too many unnecessary detail and very bad writing please stop before you do far damage the


In [21]:
# создаём новый датафрейм
df_final = pd.DataFrame()
df_final['text'] = corpus
df_final['text_lemmatized'] = corpus_lemm
df_final['label'] = df_toxic_comments.toxic
df_final.head()

,text,text_lemmatized,label
0,explanation why the edits made under my userna...,explanation why the edit make under my usernam...,0
1,d aww he matches this background colour i m se...,d aww he match this background colour I m seem...,0
2,hey man i m really not trying to edit war it s...,hey man I m really not try to edit war it s ju...,0
3,more i can t make any real suggestions on impr...,more I can t make any real suggestion on impro...,0
4,you sir are my hero any chance you remember wh...,you sir be my hero any chance you remember wha...,0


In [22]:
# делим выборку на тренировочную и тестовую
X_train, X_test, y_train, y_test = train_test_split(
    df_final.text_lemmatized,
    df_final.label,
    random_state=RANDOM_STATE,
    test_size=TEST_SIZE,
    stratify=df_final.label
)

In [23]:
# проверяем, как прошло разделение на выборки
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((119460,), (39821,), (119460,), (39821,))

**Вывод**

Для дальнейшего обучения моделей тексты были подготовлены с использованием языковой модели `spacy` и переведены в векторную форму с помощью `TfidfVectorizer`.

Выборка была разделена на тренировочную и тестовую части в соотношении 1:3 со стратификацией по целевому признаку ввиду значимого дисбаланса классов целевого признака.

Векторизация текстов будет производиться на следующем этапе в пайплайне.

## Обучение с учителем. Задача бинарной классификации

В данном разделе мы решим задачу бинарной классификации путём обучения с учителем трёх моделей:

- `CatBoostClassifier`;
- `RandomForestClassifier`;
- `LogisticRegression`.

С целью экономии вычислительных ресурсов и сокращения времени выполнения кода для каждой из моделей мы подберём по одному гиперпараметру с помощью `GridSearchCV`, кросс-валидацию выполним на трёх фолдах.

Выбор лучшей модели будет производиться по значению метрики *F1*, которое для успешно обученной модели не должно быть ниже 0.75.

In [28]:
# создаем функцию для поиска гиперпараметров
def search_best_params(hyper_param_search_method,
                       model,
                       param_grid,
                       cv=5,
                       scoring=None,
                       random_state=None,
                       n_jobs=1,
                       X_train=None,
                       y_train=None,
                       scoring_name=None,
                       hyper_param_search_method_name=None):
    # инициализируем модель для поиска лучших гиперпараметов
    if hyper_param_search_method_name == 'OptunaSearchCV':
        gs = hyper_param_search_method(
            model,
            param_grid,
            cv=cv,
            scoring=scoring,
            random_state=random_state,
            n_jobs=n_jobs
        )
    else:
        gs = hyper_param_search_method(
            model,
            param_grid,
            cv=cv,
            scoring=scoring,
            n_jobs=n_jobs
        )

    # запускаем поиск гиперпараметров
    gs.fit(X_train, y_train)

    # получаем лучшую метрику при кросс-валидации
    print(f'Лучший результат метрики {scoring_name} при кросс-валидации:\n',
          gs.best_score_)

    # получаем лучшие гиперпараметры при кросс-валидации
    print(f'Лучшие параметры модели при кросс-валидации:\n',
          gs.best_params_)
    
    return gs

In [29]:
# загружаем коллекцию стоп-слов
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Обучение модели CatBoostClassifier

In [30]:
%%time
# создаём пайплайн: счётчик слов и модель
cbc_pipe = Pipeline(
    [
        (
            'vect', TfidfVectorizer(
            stop_words=nltk_stopwords.words('english'),
            min_df=0.0001
            )
        ),
        (
            'cbc', CatBoostClassifier(
                iterations=100,
                random_state=RANDOM_STATE,
                logging_level='Silent'
            )
        )
    ]
)

# задаем гиперпараметры для перебора
param_grid = {
    'cbc__learning_rate': [0.3, 0.7, 1]
}

# выполняем поиск гиперпараметров
gs = search_best_params(GridSearchCV,
                        cbc_pipe,
                        param_grid,
                        cv=3,
                        scoring='f1',
                        n_jobs=-1,
                        X_train=X_train,
                        y_train=y_train.values,
                        scoring_name='F1')

# обучаем лучшую модель
cbc_best_model = gs.best_estimator_.fit(X_train, y_train.values)

Лучший результат метрики F1 при кросс-валидации:
 0.749929776172176
Лучшие параметры модели при кросс-валидации:
 {'cbc__learning_rate': 0.7}
CPU times: user 26min 20s, sys: 13.7 s, total: 26min 34s
Wall time: 26min 50s


Лучшей моделью `CatBoostClassifier` является модель с гиперпараметром `learning_rate` = 0.7, значение метрики *F1* которой при кросс-валидации не превысило порог 0.75. Данная модель не может быть использована для выполнения предсказаний на тестовой выборке.

### Обучение модели RandomForestClassifier

In [31]:
%%time
# создаём пайплайн: счётчик слов и модель
rfc_pipe = Pipeline(
    [
        (
            'vect', TfidfVectorizer(
            stop_words=nltk_stopwords.words('english'),
            min_df=0.0001
            )
        ),
        (
            'rfc', RandomForestClassifier(
                random_state=RANDOM_STATE,
                n_jobs=-1
            )
        )
    ]
)

# задаем гиперпараметры для перебора
param_grid = {
    'rfc__max_depth': [100, 300, 500]
}

# выполняем поиск гиперпараметров
gs = search_best_params(GridSearchCV,
                        rfc_pipe,
                        param_grid,
                        cv=3,
                        scoring='f1',
                        n_jobs=-1,
                        X_train=X_train,
                        y_train=y_train.values,
                        scoring_name='F1')

# обучаем лучшую модель
rfc_best_model = gs.best_estimator_.fit(X_train, y_train.values)

Лучший результат метрики F1 при кросс-валидации:
 0.7538171364497998
Лучшие параметры модели при кросс-валидации:
 {'rfc__max_depth': 500}
CPU times: user 26min 7s, sys: 9.23 s, total: 26min 16s
Wall time: 26min 32s


Лучшей моделью `RandomForestClassifier` является модель с гиперпараметром `max_depth` = 500, значение метрики *F1* которой при кросс-валидации превысило порог 0.75. Данная модель может быть использована для выполнения предсказаний на тестовой выборке.

### Обучение модели логистической регрессии

In [32]:
%%time
# создаём пайплайн: счётчик слов и модель
lrc_pipe = Pipeline(
    [
        (
            'vect', TfidfVectorizer(
            stop_words=nltk_stopwords.words('english'),
            min_df=0.0001
            )
        ),
        (
            'lrc', LogisticRegression(
                random_state=RANDOM_STATE,
                n_jobs=-1
            )
        )
    ]
)

# задаем гиперпараметры для перебора
param_grid = {
    'lrc__C': [0.1, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100, 1000]
}

# выполняем поиск гиперпараметров
gs = search_best_params(GridSearchCV,
                        lrc_pipe,
                        param_grid,
                        cv=3,
                        scoring='f1',
                        n_jobs=-1,
                        X_train=X_train,
                        y_train=y_train.values,
                        scoring_name='F1')

# обучаем лучшую модель
lrc_best_model = gs.best_estimator_.fit(X_train, y_train.values)

Лучший результат метрики F1 при кросс-валидации:
 0.7767446728889916
Лучшие параметры модели при кросс-валидации:
 {'lrc__C': 9}
CPU times: user 7min 37s, sys: 3min 52s, total: 11min 29s
Wall time: 11min 33s


Лучшей моделью `LogisticRegression` является модель с L2-регуляризацией и гиперпараметром `С` = 9, значение метрики *F1* которой при кросс-валидации превысило порог 0.75. Данная модель может быть использована для выполнения предсказаний на тестовой выборке.

### Проверка лучшей модели на тестовой выборке

In [33]:
# получаем предсказания с помощью лучшей модели
predictions = lrc_best_model.predict(X_test)
print('Значение метрики F1 лучшей модели на тестовой выборке:',
      f1_score(y_test, predictions))

Значение метрики F1 лучшей модели на тестовой выборке: 0.7859486057955167


**Вывод**

По результатам обучения лучшей из всех моделей является модель `LogisticRegression` с L2-регуляризацией и гиперпараметром `C` = 9, показавшая значение метрики *F1* на кросс-валидации 0.78.

На тестовой выборке значение метрики *F1* для модели `LogisticRegression` составило 0.79, то есть качество модели на тестовой выборе не ухудшилось по сравнению с качеством модели при кросс-валидации, что говорит об отсутствии переобученности и хорошей предсказательной способности модели без дополнительной настройки вероятностного порога классификации.

## Общий вывод

В исследовании были изучены около 160 тыс. текстов комментариев пользователей, размеченных следующим образом:

- `0` - нетоксичный комментарий;
- `1` - токсичный комментарий.

В выборке наблюдается существенный дисбаланс классов целевого признака: 90% комментариев относятся к классу `0` (нетоксичные), 10% - к классу `1` (токсичные).

На этапе предобработки тексты были очищены от незначимых символов с помощью регулярных выражений, и из выборки были были удалены 11 строк ввиду отсутствия буквенных текстов в них, что не повлияло на репрезентативность выборки после удаления.

Тексты были токенизированы и лемматизированы с использованием языковой модели `spacy` и переведены в векторную форму с помощью `TfidfVectorizer` в пайплайне.

Выборка была разделена на тренировочную и тестовую части в соотношении 1:3 со стратификацией по целевому признаку ввиду значимого дисбаланса классов целевого признака.

В исследовании была решена задача бинарной классификации и были обучены три модели (обучение с учителем):

- `CatBoostClassifier`: лучшей моделью является модель с гиперпараметром `learning_rate` = 0.7, значение метрики *F1* которой при кросс-валидации составило 0.7499, что ниже требуемого порога 0.75;
- `RandomForestClassifier`: лучшей моделью является модель с гиперпараметром `max_depth` = 500, значение метрики *F1* которой при кросс-валидации составило 0.754, что выше требуемого порога 0.75;
- `LogisticRegression`: лучшей моделью является модель с L2-регуляризацией и гиперпараметром `C` = 9, значение метрики *F1* которой при кросс-валидации составило 0.78, что выше требуемого порога 0.75.

По результатам обучения лучшей из всех моделей была признана модель `LogisticRegression` с L2-регуляризацией и гиперпараметром `C` = 9, показавшая значение метрики *F1* на тестовой выборке 0.79, что выше требуемого порога 0.75 и не хуже значения метрики *F1* данной модели при кросс-валидации.

Таким образом, для классификации комментариев пользователей рекомендуется использовать модель `LogisticRegression` с L2-регуляризацией и гиперпараметром `C` = 9 для признаков, подготовленных вышеописанным образом.

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны